## Example: Initialization and Primitives of the Executor

This example shows different ways to initialize the Executor and run various jobs.

In [1]:
from qiskit_aer import Aer
from qiskit.circuit.random import random_circuit
from qiskit.primitives import Sampler, Estimator, BackendSampler, BackendEstimator
from qiskit.quantum_info import SparsePauliOp
from qiskit_ibm_runtime import QiskitRuntimeService, Session
from qiskit_ibm_runtime import Estimator as RuntimeEstimator

from squlearn.util import Executor
from squlearn.util.executor import BaseEstimatorV1, BaseSamplerV1

## Executor Initialization with Local Simulators

The following cells show different ways to initialize the Executor with local simulators.

In [2]:
# from a string representing the simple Qiskit Aer simulators:
executor = Executor("statevector_simulator")
executor = Executor("qasm_simulator")

# from a backend following the Qiskit backend standard:
executor = Executor(Aer.get_backend("aer_simulator_statevector"))

## Executor Initialization with Qiskit Primitives

In [3]:
# from a Qiskit simulator primitive:
executor = Executor(Estimator())
executor = Executor(Sampler())
executor = Executor(BackendEstimator(Aer.get_backend("aer_simulator")))
executor = Executor(BackendSampler(Aer.get_backend("aer_simulator")))

C:\Users\mow\AppData\Local\Temp\ipykernel_28608\3565786267.py:2: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  executor = Executor(Estimator())
C:\Users\mow\AppData\Local\Temp\ipykernel_28608\3565786267.py:3: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  executor = Executor(Sampler())
C:\Users\mow\AppData\Local\Temp\ipykernel_28608\3565786267.py:4: DeprecationWarning: The class ``qiski

## IBM Quantum Backend Initialization

When using IBM Quantum backends, always use a context manager (``with`` statement) to ensure that sessions 
are properly closed and you are not charged for unused sessions.

### Example 1: IBM Backend with Context Manager (Recommended)

In [ ]:
# RECOMMENDED: Using context manager for proper session management
service = QiskitRuntimeService(channel="ibm_quantum_platform")
# Alternative: service = QiskitRuntimeService(channel="ibm_quantum_platform", token="YOUR_TOKEN_HERE")

with Executor(service.backend("ibm_kingston")) as executor:
    print(f"Using backend: {executor.backend_name}")
    # Your code here
# Session is automatically closed

Using backend: ibm_kingston


### Example 2: Pre-initialized Session with Context Manager (Recommended)

In [5]:
service = QiskitRuntimeService(channel="ibm_quantum_platform")

with Executor(service.backend("ibm_kingston")) as executor:
    print(f"Backend: {executor.backend_name}")
    if executor.session:
        print(f"Session ID: {executor.session.session_id}")
    # Your code here
# Session is automatically closed

Backend: ibm_kingston


### Example 3: IBM Quantum Estimator with Context Manager (Recommended)

In [6]:
service = QiskitRuntimeService(channel="ibm_quantum_platform")
session = Session(backend=service.backend("ibm_kingston"), max_time=28800)
runtime_estimator = RuntimeEstimator(session)

with Executor(runtime_estimator) as executor:
    print(f"Backend: {executor.backend_name}")
    # Your code here
# Session is automatically closed

Backend: ibm_kingston


### Example 4: Manual Session Closing (Alternative)

If you cannot use a context manager, you **must** manually close the session when done to avoid unexpected charges.
However, using a context manager is strongly recommended.

In [8]:
service = QiskitRuntimeService(channel="ibm_quantum_platform")
executor = Executor(service.backend("ibm_kingston"))

try:
    print(f"Backend: {executor.backend_name}")
    executor.create_session()
    # Your code here
finally:
    executor.close_session()
    print("Session manually closed")

Backend: ibm_kingston


C:\Users\mow\Documents\dev\squlearn\src\squlearn\util\executor.py:2476: SessionContextMisuseWarning: Creating a session outside of a context manager may lead to  unclosed sessions. It is recommended to use the Executor within a 'with' statement. At least make sure to call 'executor.close_session()' when you are done with the executor or make sure it is properly garbage collected.
  warnings.warn(


Session manually closed


## Setting Shots

The following cell demonstrates how to set the number of shots utilized in the circuit evaluation.

In [9]:
# Shots can be set by the executor:
print("Current shots as set before:", executor.get_shots())

# Set shots
executor.set_shots(1234)
print("Adjusted shots:", executor.get_shots())

# Reset shots to initial ones:
executor.reset_shots()
print("Reset shots:", executor.get_shots())

Current shots as set before: 4000
Adjusted shots: 1234
Reset shots: 4000


## Using Executor Primitives

In this cell, we calculate an expectation value using the Estimator primitive, which is accessible through the ``Executor`` class. 
The executor generates modified Primitives with enhanced functionality, including caching, automatic session management, and logging capabilities.
These modified primitives can be seamlessly incorporated into your workflow or other Qiskit routines.

In [10]:
# Generate a random circuit:
circuit = random_circuit(2, 2, seed=0).decompose(reps=1)

# Generate an observable:
observable = SparsePauliOp("ZI")

# Initialize executor with local simulator
executor = Executor("statevector_simulator")

# Get the Executor Estimator Primitive and call run:
estimator = executor.get_estimator()
if isinstance(estimator, BaseEstimatorV1):
    print(estimator.run(circuit, observable, shots=4321).result())
else:
    print(estimator.run([(circuit, observable)], precision=1 / 4321**0.5).result())

# Get the Executor Sampler Primitive and call run:
sampler = executor.get_sampler()
if isinstance(sampler, BaseSamplerV1):
    print(sampler.run(circuit.measure_all(inplace=False)).result())
else:
    print(sampler.run([(circuit.measure_all(inplace=False),)]).result())

PrimitiveResult([PubResult(data=DataBin(evs=np.ndarray(<shape=(), dtype=float64>), stds=np.ndarray(<shape=(), dtype=float64>)), metadata={'target_precision': 0.01521275485262189, 'circuit_metadata': {}})], metadata={'version': 2})
PrimitiveResult([SamplerPubResult(data=DataBin(meas=BitArray(<shape=(), num_shots=1024, num_bits=2>)), metadata={'shots': 1024, 'circuit_metadata': {}})], metadata={'version': 2})


## Backend Execution

The executor can also be used to execute ``backend.run()``. However, caching is not yet fully implemented for this case.

In [11]:
executor = Executor("qasm_simulator")
job = executor.qiskit_execute(circuit)
job.result()

Result(backend_name='aer_simulator', backend_version='0.17.2', qobj_id='', job_id='e29cbd7e-ad0e-4169-b300-0e4036903372', success=True, results=[ExperimentResult(shots=1024, success=True, meas_level=2, data=ExperimentResultData(), header=QobjExperimentHeader(creg_sizes=[], global_phase=0.0, memory_slots=0, n_qubits=2, name='circuit-162', qreg_sizes=[['q', 2]], metadata={}), status=DONE, seed_simulator=4162277737, metadata={'batched_shots_optimization': False, 'required_memory_mb': 0, 'method': 'stabilizer', 'active_input_qubits': [], 'device': 'CPU', 'remapped_qubits': False, 'num_qubits': 0, 'num_clbits': 0, 'time_taken': 0.0002513, 'input_qubit_map': [], 'max_memory_mb': 16135, 'measure_sampling': False, 'parallel_shots': 1, 'parallel_state_update': 12, 'runtime_parameter_bind': False, 'num_bind_params': 1}, time_taken=0.0002513)], date=2026-01-21T15:24:12.886139, status=COMPLETED, header=None, metadata={'omp_enabled': True, 'parallel_experiments': 1, 'max_memory_mb': 16135, 'max_gpu

## Logging

Additionally, the executor maintains a detailed log of background operations. 
This feature proves especially valuable when optimizing real backends, 
allowing you to gain insights into the underlying processes and activities.

In [12]:
executor = Executor("qasm_simulator", log_file="example_log.log")
executor.set_shots(1234)
estimator = executor.get_estimator()

# Depending on the Estimator version, the run call is slightly different
if isinstance(estimator, BaseEstimatorV1):
    print(estimator.run(circuit, observable, shots=4321).result())
else:
    print(estimator.run([(circuit, observable)], precision=1 / 4321**0.5).result())

PrimitiveResult([PubResult(data=DataBin(evs=np.ndarray(<shape=(), dtype=float64>), stds=np.float64(0.0)), metadata={'target_precision': 0.01521275485262189, 'shots': 4321, 'circuit_metadata': {}})], metadata={'version': 2})


## Caching

The executor has a cache where it stores and can reuse job results. 
In this example, we change the number of shots to distinguish the first and second runs of the same job, 
and both runs are stored in separate caches. However, the third job simply reuses the cached result from its first execution.

In [13]:
executor = Executor(
    BackendSampler(Aer.get_backend("qasm_simulator")),
    log_file="example_log_cache.log",
    caching=True,
    cache_dir="_cache",
)
executor.set_shots(4321)
estimator = executor.get_estimator()

# Depending on the Estimator version, the run call is slightly different
if isinstance(estimator, BaseEstimatorV1):
    print(estimator.run(circuit, observable).result())
else:
    print(estimator.run([(circuit, observable)]).result())

executor.set_shots(1234)
# Depending on the Estimator version, the run call is slightly different
if isinstance(estimator, BaseEstimatorV1):
    print(estimator.run(circuit, observable).result())
else:
    print(estimator.run([(circuit, observable)]).result())

# This one is loaded from the cache and not executed again
executor.set_shots(4321)
# Depending on the Estimator version, the run call is slightly different
if isinstance(estimator, BaseEstimatorV1):
    print(estimator.run(circuit, observable).result())
else:
    print(estimator.run([(circuit, observable)]).result())

PrimitiveResult([PubResult(data=DataBin(evs=np.ndarray(<shape=(), dtype=float64>), stds=np.float64(0.0)), metadata={'target_precision': 0.01521275485262189, 'shots': 4321, 'circuit_metadata': {}})], metadata={'version': 2})
PrimitiveResult([PubResult(data=DataBin(evs=np.ndarray(<shape=(), dtype=float64>), stds=np.float64(0.0)), metadata={'target_precision': 0.028467047115478598, 'shots': 1235, 'circuit_metadata': {}})], metadata={'version': 2})
PrimitiveResult([PubResult(data=DataBin(evs=np.ndarray(<shape=(), dtype=float64>), stds=np.float64(0.0)), metadata={'target_precision': 0.01521275485262189, 'shots': 4321, 'circuit_metadata': {}})], metadata={'version': 2})


C:\Users\mow\AppData\Local\Temp\ipykernel_28608\735415858.py:2: DeprecationWarning: The class ``qiskit.primitives.backend_sampler.BackendSampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `BackendSampler` class is `BackendSamplerV2`.
  BackendSampler(Aer.get_backend("qasm_simulator")),
